In [1]:
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string

이상한 나라의 엘리스 소설을 읽어온다.

In [2]:
with open('alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()

content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
 
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

Trigram list

In [ ]:
N = 3
quads = list(nltk.ngrams(tokens, N))

In [6]:
quads[:3]

[('alice', 'adventures', 'in'),
 ('adventures', 'in', 'wonderland'),
 ('in', 'wonderland', 'alice')]

In [8]:
newl_app = []
for ln in quads:
    newl = " ".join(ln)
    newl_app.append(newl)

In [11]:
newl_app[:3]

['alice adventures in', 'adventures in wonderland', 'in wonderland alice']

Vectorizing the words

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])  # trigram 중 앞 부분 2 단어. 
    y_str = l.split()[N-1]              # trigram 중 마지막 1 단어.  
    x_trigm.append(x_str)
    y_trigm.append(y_str)

x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

Dictionaries from word to integer and integer to word

In [22]:
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}

X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)

Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)

print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


Model Building

In [23]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 10

input_layer = Input(shape = (Xtrain.shape[1],))
first_layer = Dense(1000, activation='relu')(input_layer)
first_dropout = Dropout(rate = 0.5)(first_layer)

second_layer = Dense(800, activation='relu')(first_dropout)

third_layer = Dense(1000, activation='relu')(second_layer)
third_dropout = Dropout(rate = 0.5)(third_layer)

fourth_layer = Dense(Ytrain.shape[1], activation='softmax',name = "fourth")(third_dropout)


history = Model(input_layer, fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy", metrics=["accuracy"])

print (history.summary())

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2559)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2560000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 800)               800800    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              801000    
_________________________________________________________________
dropout_2 (Dropout)  

Model Training

In [63]:
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Train on 14357 samples, validate on 3590 samples
Epoch 1/10
14357/14357 [==============================] - 24s 2ms/step - loss: 2.5097 - acc: 0.4333 - val_loss: 7.4453 - val_acc: 0.1031
Epoch 2/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.4176 - acc: 0.4387 - val_loss: 7.5718 - val_acc: 0.1019
Epoch 3/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.3286 - acc: 0.4596 - val_loss: 7.5107 - val_acc: 0.1064
Epoch 4/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.2514 - acc: 0.4679 - val_loss: 7.7205 - val_acc: 0.1050
Epoch 5/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.1901 - acc: 0.4750 - val_loss: 7.7672 - val_acc: 0.1058
Epoch 6/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.1134 - acc: 0.4888 - val_loss: 7.7929 - val_acc: 0.1070
Epoch 7/10
14357/14357 [==============================] - 23s 2ms/step - loss: 2.0519 - acc: 0.4971 - val_loss: 7.8076 - val_acc:

Model Prediction

In [ ]:
Y_pred = history.predict(Xtest)

Sample check on Test data

In [64]:
print ("Prior bigram words","| Actual","| Predicted","\n")

for i in range(10):
    print (i,xtest_tg[i], "|", rev_dictnry[np.argmax(Ytest[i])], "| ", end='')
    idx = np.flipud(Y_pred[i].argsort())[:5]
    for k in idx:
        print(rev_dictnry[k], ' ', end='')
    print()

Prior bigram words | Actual | Predicted 

0 the evening | beautiful | beautiful  turtle  rabbit  hall  little  
1 slipped in | like | and  in  the  with  half  
2 alice swallowing | down | good  her  very  little  not  
3 an encouraging | tone | of  to  but  again  voice  
4 waistcoat pocket | or | went  music  and  sat  repeated  
5 she went | on | on  down  back  hunting  up  
6 that she | knew | was  had  would  got  said  
7 down on | her | one  the  his  their  her  
8 dormouse went | on | on  down  after  without  back  
9 soup soup | of | you  said  and  to  in  
